In [79]:
import random
#import re
from datetime import date
#from urllib.request import urlretrieve
from IPython.display import clear_output
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
#from selenium.webdriver.common.by import By
from lxml import etree
import json
import time
import traceback
import os
from tqdm import tqdm

In [80]:
class crawling:
    
    def __init__(self,):
        self.home_page = "https://batdongsan.so/nha-dat-ban/ha-noi#/"
        self.root = "https://batdongsan.so/nha-dat/"
        self.page = 1
    
    def get_pages(self, page_source):
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        links = [i.getchildren()[0].get("href") for i in page.xpath("""//*[@id="main"]/div/div/section/div/div[1]""")[0].getchildren()]
        links = [link for link in links if link != None]
        return links
    
    def next_page(self):
        self.page += 1
        return self.home_page.removesuffix('#/') + """?page=""" + f"{self.page}" + "#/"
    
    def gather(self, page_source):
        data = {}
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        #get price
        price_path=page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[1]/div[2]/div[2]/strong""")
        if len(price_path)!=0:
            data["price"]=page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[1]/div[2]/div[2]/strong""")[0].text + " " + page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[1]/div[2]/div[2]/text()[2]""")[0].strip()
        else:
            data["price"]=None
        
        #get address
        if len(bs.find_all("div", {"class": "re-address"}))!=0:
            address = bs.find_all("div", {"class": "re-address"})[0].text.strip()
            data["address"] = address
        else:
            data["address"] = None

        
        #get detailed description
        des_elements = list(bs.find_all("ul", {"class": "re-property"})[0].children)[0:-1:2]
        for i in range(len(des_elements)):
            key_value = des_elements[i].text.strip().split(":")
            key = key_value[0]
            value = key_value[1]
            data[key] = value
        
        #get more information
        info = bs.find("div", {"class": "re-content"}).text
        data["Details"] = info
        
        #get id
        other_information= list(bs.find_all("ul", {"class": "short-detail-2 list2 clearfix"})[0].children)[0:-1:2]
        try:
            for i in range(len(other_information)):
                key_value = other_information[i].text.strip().split(":")
                key = key_value[0]
                value = key_value[1]
                data[key] = value
            if "Mã tin" in data:
                data["id"] = "batdongsan_so_com_vn_" + str(data["Mã tin"])
            else:
                data["id"] = "batdongsan_so_com_vn_" + str(time.time())
        except Exception as e:
            print(e)
            import pdb
            pdb.set_trace()
        return data
    
    def run(self, start, num_of_pages):
        service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
        driver = webdriver.Edge(service=service)
        url = self.home_page
        self.page = 1
        if start != 1:
            self.page = start
            url = self.home_page.removesuffix('#/') + """?page=""" + f"{self.page}" + "#/"
        if os.path.exists("data/index.json"):
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
            
        with open("log.txt", "w") as f:
            pass
        
        dataset = []
        for i in range(num_of_pages):
            num = 10
            pages = []
            while num > 0:
                try:
                    driver.get(url)
                    time.sleep(0.05)
                    pages = self.get_pages(driver.page_source)
                    driver.delete_all_cookies()
                    break
                except:
                    num -= 1
            if num <= 0:
                url = self.next_page()
                continue
            #driver.close()
            #service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
            #driver = webdriver.Edge(service=service)
            for page in tqdm(pages):
                num = 10
                while num > 0:
                    try:
                        driver.get(page)
                        time.sleep(0.05)
                        data = self.gather(driver.page_source)
                        if data["id"] not in index:
                            dataset.append(data)
                            index.add(data["id"])
                        with open("log.txt", "a") as f:
                            f.write("Success:" + page +"\n")
                        driver.delete_all_cookies()
                        break
                    except Exception as e:
                        print(e)
                        with open("log.txt", "a") as f:
                            f.write(f"Excution{10-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        driver.delete_all_cookies()
                        num -= 1
            url = self.next_page()
        if len(dataset) != 0:
            df = pd.DataFrame(dataset)
            df.to_csv(f"data/{start}_{start+num_of_pages}_{time.time()}.csv", index=False)
        with open("data/index.json", "w") as f:
            json.dump(list(index), f)
        driver.close()
        

In [81]:
craw = crawling()

In [ ]:
for i in tqdm(range(1, 3227)):
    craw.run(i, 1)
    clear_output()

In [23]:
service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
driver = webdriver.Edge(service=service)

In [75]:
page_source = "https://batdongsan.so/nha-dat/ban-nha-mat-pho-hoang-mai-62m2-7-tang-thang-may-kd-mt-5m-chi-15-8-ty#/"
driver.get(page_source)
#print(bs)
page = etree.HTML(str(bs))
print(page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[2]/div[3]""")[0].getchildren())

[<Element br at 0x23e30e17580>, <Element br at 0x23e32a33bc0>, <Element br at 0x23e328f2dc0>, <Element br at 0x23e3231f780>, <Element br at 0x23e2fbf1840>, <Element br at 0x23e2fbf2440>, <Element br at 0x23e2fbf3ac0>, <Element br at 0x23e2fbf2980>, <Element br at 0x23e2fbf3240>, <Element br at 0x23e2fbf14c0>, <Element br at 0x23e2fbf3280>, <Element br at 0x23e2fbf2180>, <Element br at 0x23e2fbf2240>, <Element br at 0x23e2fbf1000>, <Element br at 0x23e2fbf0fc0>, <Element br at 0x23e2fbf20c0>, <Element br at 0x23e2fbf2b00>, <Element br at 0x23e2fbf3140>, <Element br at 0x23e2fbf1680>, <Element br at 0x23e2fbf3980>, <Element br at 0x23e2fbf2800>, <Element br at 0x23e2fbf2480>]


In [78]:
page_source = "https://batdongsan.so/nha-dat/ban-nha-pho-ngoc-khanh-ba-dinh-tang-noi-that-o-ngay-35m2-4-tang-gia-3-85-ty#/"
driver.get(page_source)
bs = BeautifulSoup(driver.page_source)
result = bs.find("div", {"class": "re-content"})
print(result.text)


+ Chủ nhà anh Toàn đã mua nhà mới, cần bán gấp nhà phố Ngọc Khánh - Ba Đình - Hà Nội. Diện tích: 35m2 - 4 Tầng - Mặt tiền 4m. Khu dân trí cao, an ninh tốt.+ Vị trí đẹp, ngõ rộng thênh thang, vào bước chân ra ô tô tránh. An sinh đỉnh, gần Hồ Giảng Võ, bãi gửi ô tô, trường học, bệnh viện Xanh Pôn, ga tàu điện ngầm, chợ, ngân hàng, gần nhiều nhà hàng, quán cafe.+ Giao thông thuận tiện kết nối phố Ngọc Khánh - Giảng Võ - Kim Mã - Nguyễn Công Hoan - Nguyễn Chí Thanh - Láng Hạ - La Thành.+ Nhà chủ tự xây 4 tầng chắc chắn, thiết kế hiện đại, đầy đủ công năng.+ Tặng lại toàn bộ nội thất cho khách thiện chí.+ Sổ đỏ chính chủ sẵn sàng giao dịch.+ Giá: 3.85 tỷ ( có thương lượng ).Liên hệ Mr Liệu để xem nhà miễn phí, làm việc chính chủ, miễn trung gian.
